In [1]:
!curl -I https://www.google.com


HTTP/2 200 
content-type: text/html; charset=ISO-8859-1
content-security-policy-report-only: object-src 'none';base-uri 'self';script-src 'nonce-ClNiUCoYkzDhnJXTw4pkYA' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-inline' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp
accept-ch: Sec-CH-Prefers-Color-Scheme
p3p: CP="This is not a P3P policy! See g.co/p3phelp for more info."
date: Thu, 17 Apr 2025 13:55:31 GMT
server: gws
x-xss-protection: 0
x-frame-options: SAMEORIGIN
expires: Thu, 17 Apr 2025 13:55:31 GMT
cache-control: private
set-cookie: AEC=AVcja2cW54R-PXB6wZficpfCenABmVolhLhANZsPfQQo71G_XQ6lJ35GUA; expires=Tue, 14-Oct-2025 13:55:31 GMT; path=/; domain=.google.com; Secure; HttpOnly; SameSite=lax
set-cookie: NID=523=MYpSX-bJz6peOhAxvGNbAcVBzxyeNAnxMzRKkvi3zHMokj3lVcUWlibMJS6EWTRN_Mfc27vj4IUkD2uWdmTLFV90n-wAFaI5vGOGmdrk8tZ1BX-0LgejDruabug1NZYsHo1gts7NjJSE91TxRFSuUjZlPcp0a64wPB56aebM72q_rwbsrJiFn5POylCI6E_lYaIJcNJwv2BNwXPN2-9q; expires=Fri, 17-Oct-2025 13

In [2]:
!wget -c http://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz
!gunzip -c hg19.fa.gz > hg19.fa


--2025-04-17 13:57:03--  http://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 948731419 (905M) [application/x-gzip]
Saving to: ‘hg19.fa.gz’

hg19.fa.gz          100%[===================>] 904.78M  37.9MB/s    in 25s     

2025-04-17 13:57:29 (36.4 MB/s) - ‘hg19.fa.gz’ saved [948731419/948731419]



In [4]:
!ls -lh

total 3.9G
-rw-r--r-- 1 sagemaker-user users 7.9K Apr 17 13:54 README.md
-rw-r--r-- 1 sagemaker-user users 4.1K Apr 17 13:59 Untitled.ipynb
-rw-r--r-- 1 sagemaker-user users  77K Apr 17 13:54 getting_started.ipynb
-rw-r--r-- 1 sagemaker-user users 3.0G Apr 17 13:58 hg19.fa
-rw-r--r-- 1 sagemaker-user users 905M Aug 21  2018 hg19.fa.gz


In [5]:
import re
import csv

# Write a sample FASTA file using Python code.
def generate_example_fasta(filename="example.fasta"):
    fasta_content = (
        ">seq1\n"
        "CCAAAAAAATAAAAAAAAAAAAGG\n"
        ">seq2\n"
        "CTTTTTTTTATTTTTTTTTTTTAG\n"
    )
    with open(filename, "w") as f:
        f.write(fasta_content)
    print(f"FASTA file '{filename}' has been generated.")

def reverse_complement(seq):
    """Return the reverse complement of a DNA sequence."""
    complement = str.maketrans("ACGTacgt", "TGCAtgca")
    return seq.translate(complement)[::-1]

def parse_fasta(fasta_file):
    """A simple FASTA parser. Returns a dict {header: sequence}."""
    sequences = {}
    header = None
    seq_lines = []
    with open(fasta_file, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if header:
                    sequences[header] = "".join(seq_lines)
                header = line[1:].split()[0]
                seq_lines = []
            else:
                seq_lines.append(line)
        if header:
            sequences[header] = "".join(seq_lines)
    return sequences

def find_sgrna_binding_sites(fasta_file):
    pattern = re.compile(r'(?=(([ACGT]{20}[ACGT](GG|AG))))', re.IGNORECASE)

    results = []
    sequences = parse_fasta(fasta_file)

    for seq_id, seq in sequences.items():
        seq = seq.upper()

        # Positive strand
        for match in re.finditer(pattern, seq):
            start = match.start()
            end = start + 23
            results.append({
                "seq_id": seq_id,
                "strand": "+",
                "start": start+1,
                "end": end,
                "target": seq[start:end]
            })

        # Negative strand
        rc_seq = reverse_complement(seq)
        for match in re.finditer(pattern, rc_seq):
            rc_start = match.start()
            rc_end = rc_start + 23
            orig_end = len(seq) - rc_start
            orig_start = len(seq) - rc_end
            results.append({
                "seq_id": seq_id,
                "strand": "-",
                "start": orig_start+1,
                "end": orig_end,
                "target": rc_seq[rc_start:rc_end]
            })

    return results

def export_to_csv(results, fasta_file):
    output_file = fasta_file + ".csv"
    with open(output_file, "w", newline='') as csvfile:
        fieldnames = ["seq_id", "strand", "start", "end", "target"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for entry in results:
            writer.writerow(entry)
    print(f"Results written to {output_file}")

# Example usage
fasta_file = "example.fasta"
generate_example_fasta(fasta_file)
sites = find_sgrna_binding_sites(fasta_file)
export_to_csv(sites, fasta_file)


FASTA file 'example.fasta' has been generated.
Results written to example.fasta.csv


In [7]:
!grep "^>" hg19.fa


>chr1
>chr2
>chr3
>chr4
>chr5
>chr6
>chr7
>chrX
>chr8
>chr9
>chr10
>chr11
>chr12
>chr13
>chr14
>chr15
>chr16
>chr17
>chr18
>chr20
>chrY
>chr19
>chr22
>chr21
>chr6_ssto_hap7
>chr6_mcf_hap5
>chr6_cox_hap2
>chr6_mann_hap4
>chr6_apd_hap1
>chr6_qbl_hap6
>chr6_dbb_hap3
>chr17_ctg5_hap1
>chr4_ctg9_hap1
>chr1_gl000192_random
>chrUn_gl000225
>chr4_gl000194_random
>chr4_gl000193_random
>chr9_gl000200_random
>chrUn_gl000222
>chrUn_gl000212
>chr7_gl000195_random
>chrUn_gl000223
>chrUn_gl000224
>chrUn_gl000219
>chr17_gl000205_random
>chrUn_gl000215
>chrUn_gl000216
>chrUn_gl000217
>chr9_gl000199_random
>chrUn_gl000211
>chrUn_gl000213
>chrUn_gl000220
>chrUn_gl000218
>chr19_gl000209_random
>chrUn_gl000221
>chrUn_gl000214
>chrUn_gl000228
>chrUn_gl000227
>chr1_gl000191_random
>chr19_gl000208_random
>chr9_gl000198_random
>chr17_gl000204_random
>chrUn_gl000233
>chrUn_gl000237
>chrUn_gl000230
>chrUn_gl000242
>chrUn_gl000243
>chrUn_gl000241
>chrUn_gl000236
>chrUn_gl000240
>chr17_gl000206_random
>chrUn_gl000

In [8]:
%%writefile keep.txt
chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX


Writing keep.txt


In [9]:
!sudo apt-get install -y seqtk
!seqtk subseq hg19.fa keep.txt > hg19_main.fa


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  seqtk
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 30.2 kB of archives.
After this operation, 85.0 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 seqtk amd64 1.3-2 [30.2 kB]
Fetched 30.2 kB in 0s (313 kB/s) 
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package seqtk.
(Reading database ... 14526 files and directories currently installed.)
Preparing to unpack .../archives/seqtk_1.3-2_amd64.deb ...
Unpacking seqtk (1.3-2) ...
Setting up seqtk (1.3-2) ...


In [10]:
!grep "^>" hg19_main.fa

>chr1
>chr2
>chr3
>chr4
>chr5
>chr6
>chr7
>chrX
>chr8
>chr9
>chr10
>chr11
>chr12
>chr13
>chr14
>chr15
>chr16
>chr17
>chr18
>chr20
>chr19
>chr22
>chr21


In [1]:
import re
import csv

def reverse_complement(seq):
    """Return the reverse complement of a DNA sequence."""
    complement = str.maketrans("ACGTacgt", "TGCAtgca")
    return seq.translate(complement)[::-1]

def parse_fasta(fasta_file):
    """A simple FASTA parser. Returns a dict {header: sequence}."""
    sequences = {}
    header = None
    seq_lines = []
    with open(fasta_file, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            if line.startswith(">"):
                if header:
                    sequences[header] = "".join(seq_lines)
                header = line[1:].split()[0]
                seq_lines = []
            else:
                seq_lines.append(line)
        if header:
            sequences[header] = "".join(seq_lines)
    return sequences

def find_sgrna_binding_sites(fasta_file):
    pattern = re.compile(r'(?=(([ACGT]{20}[ACGT](GG|AG))))', re.IGNORECASE)

    results = []
    sequences = parse_fasta(fasta_file)

    for seq_id, seq in sequences.items():
        seq = seq.upper()

        # Positive strand
        for match in re.finditer(pattern, seq):
            start = match.start()
            end = start + 23
            results.append({
                "seq_id": seq_id,
                "strand": "+",
                "start": start+1,
                "end": end,
                "target": seq[start:end]
            })

        # Negative strand
        rc_seq = reverse_complement(seq)
        for match in re.finditer(pattern, rc_seq):
            rc_start = match.start()
            rc_end = rc_start + 23
            orig_end = len(seq) - rc_start
            orig_start = len(seq) - rc_end
            results.append({
                "seq_id": seq_id,
                "strand": "-",
                "start": orig_start+1,
                "end": orig_end,
                "target": rc_seq[rc_start:rc_end]
            })

    return results

def export_to_csv(results, fasta_file):
    output_file = fasta_file + ".csv"
    with open(output_file, "w", newline='') as csvfile:
        fieldnames = ["seq_id", "strand", "start", "end", "target"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for entry in results:
            writer.writerow(entry)
    print(f"Results written to {output_file}")
#################################################################################
import os
from pathlib import Path
from tqdm import tqdm

# Setup input/output paths

output_dir = "split_by_seq"
os.makedirs(output_dir, exist_ok=True)

# Count total sequences to initialize progress bar
with open('hg19_main.fa', 'r') as f:
    total_seqs = sum(1 for line in f if line.startswith('>'))

# Initialize variables
current_header = None
current_seq = []
input_path = 'hg19_main.fa'
# Process and split into separate files with progress bar
with open(input_path, 'r') as f, tqdm(total=total_seqs, desc="Splitting FASTA") as pbar:
    for line in f:
        line = line.strip()
        if line.startswith('>'):  # header line
            if current_header:
                # Save previous sequence
                filename = current_header[1:].split()[0]  # clean filename
                output_path = os.path.join(output_dir, f"{filename}.fasta")
                with open(output_path, 'w') as out_f:
                    out_f.write(current_header + "\n")
                    out_f.write("".join(current_seq) + "\n")
                pbar.update(1)
            current_header = line
            current_seq = []
        else:
            current_seq.append(line)

    # Save last sequence
    if current_header:
        filename = current_header[1:].split()[0]
        output_path = os.path.join(output_dir, f"{filename}.fasta")
        with open(output_path, 'w') as out_f:
            out_f.write(current_header + "\n")
            out_f.write("".join(current_seq) + "\n")
        pbar.update(1)

print(f"Done. Files saved to {output_dir}")
import os
from glob import glob
from tqdm import tqdm  # Progress bar

# Change this path to where your FASTA files are stored
fasta_dir = "split_by_seq/"
fasta_files = glob(os.path.join(fasta_dir, "*.fasta"))

# Create output directory
os.makedirs("output_csvs", exist_ok=True)

# Process all FASTA files with a progress bar
for fasta_file in tqdm(fasta_files, desc="Processing FASTA files"):
    print(f"\nProcessing: {fasta_file}")
    sites = find_sgrna_binding_sites(fasta_file)
    export_to_csv(sites, fasta_file)
!ls -lh /content/split_by_seq/*.csv

Splitting FASTA: 100%|██████████| 23/23 [00:23<00:00,  1.00s/it]


Done. Files saved to split_by_seq


Processing FASTA files:   0%|          | 0/23 [00:00<?, ?it/s]


Processing: split_by_seq/chr1.fasta


Processing FASTA files:   4%|▍         | 1/23 [05:42<2:05:42, 342.85s/it]

...

Results written to split_by_seq/chr19.fasta.csv

Processing: split_by_seq/chr22.fasta


Processing FASTA files:  96%|█████████▌| 22/23 [1:10:44<01:37, 97.25s/it] 

Results written to split_by_seq/chr22.fasta.csv

Processing: split_by_seq/chr21.fasta


Processing FASTA files: 100%|██████████| 23/23 [1:11:38<00:00, 186.88s/it]

Results written to split_by_seq/chr21.fasta.csv
ls: cannot access '/content/split_by_seq/*.csv': No such file or directory


In [4]:
!ls -lh split_by_seq/*.csv

-rw-r--r-- 1 sagemaker-user users  2.7G May  5 08:49 split_by_seq/chr1.fasta.csv
-rw-r--r-- 1 sagemaker-user users  1.6G May  5 09:31 split_by_seq/chr10.fasta.csv
-rw-r--r-- 1 sagemaker-user users  1.6G May  5 09:34 split_by_seq/chr11.fasta.csv
-rw-r--r-- 1 sagemaker-user users  1.6G May  5 09:38 split_by_seq/chr12.fasta.csv
-rw-r--r-- 1 sagemaker-user users  1.1G May  5 09:40 split_by_seq/chr13.fasta.csv
-rw-r--r-- 1 sagemaker-user users  1.1G May  5 09:42 split_by_seq/chr14.fasta.csv
-rw-r--r-- 1 sagemaker-user users 1011M May  5 09:44 split_by_seq/chr15.fasta.csv
-rw-r--r-- 1 sagemaker-user users  1.1G May  5 09:46 split_by_seq/chr16.fasta.csv
-rw-r--r-- 1 sagemaker-user users  1.1G May  5 09:49 split_by_seq/chr17.fasta.csv
-rw-r--r-- 1 sagemaker-user users  850M May  5 09:50 split_by_seq/chr18.fasta.csv
-rw-r--r-- 1 sagemaker-user users  792M May  5 09:54 split_by_seq/chr19.fasta.csv
-rw-r--r-- 1 sagemaker-user users  2.8G May  5 08:55 split_by_seq/chr2.fasta.csv
-rw-r--r-- 1 sagem

In [8]:
!head split_by_seq/chr1.fasta.csv -n 10
!tail split_by_seq/chr1.fasta.csv -n 10

seq_id,strand,start,end,target
chr1,+,10451,10473,AACCCTAACCCTAACCCTCGCGG
chr1,+,10460,10482,CCTAACCCTCGCGGTACCCTCAG
chr1,+,10464,10486,ACCCTCGCGGTACCCTCAGCCGG
chr1,+,10477,10499,CCTCAGCCGGCCCGCCCGCCCGG
chr1,+,10478,10500,CTCAGCCGGCCCGCCCGCCCGGG
chr1,+,10489,10511,CGCCCGCCCGGGTCTGACCTGAG
chr1,+,10490,10512,GCCCGCCCGGGTCTGACCTGAGG
chr1,+,10492,10514,CCGCCCGGGTCTGACCTGAGGAG
chr1,+,10511,10533,GGAGAACTGTGCTCCGCCTTCAG
chr1,-,10022,10044,TAGGGTTAGGGTTAGGGTTAGGG
chr1,-,10018,10040,GTTAGGGTTAGGGTTAGGGTTAG
chr1,-,10017,10039,TTAGGGTTAGGGTTAGGGTTAGG
chr1,-,10016,10038,TAGGGTTAGGGTTAGGGTTAGGG
chr1,-,10012,10034,GTTAGGGTTAGGGTTAGGGTTAG
chr1,-,10011,10033,TTAGGGTTAGGGTTAGGGTTAGG
chr1,-,10010,10032,TAGGGTTAGGGTTAGGGTTAGGG
chr1,-,10006,10028,GTTAGGGTTAGGGTTAGGGTTAG
chr1,-,10005,10027,TTAGGGTTAGGGTTAGGGTTAGG
chr1,-,10004,10026,TAGGGTTAGGGTTAGGGTTAGGG
